In [ ]:
%pip install PyCryptodome

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.number import inverse
from hashlib import sha1
import math
from tqdm.auto import tqdm

In [ ]:
from collections import namedtuple
Point = namedtuple("Point", "x y") #from source.py

def point_addition(P, Q):
    Rx = (P.x*Q.x + D*P.y*Q.y) % p
    Ry = (P.x*Q.y + P.y*Q.x) % p
    return Point(Rx, Ry)


def scalar_multiplication(P, n):
    Q = Point(1, 0)
    while n > 0:
        if n % 2 == 1:
            Q = point_addition(Q, P)
        P = point_addition(P, P)
        n = n//2
    return Q

In [ ]:
# Curve parameters - from source.py
p = 173754216895752892448109692432341061254596347285717132408796456167143559
D = 529
G = Point(29394812077144852405795385333766317269085018265469771684226884125940148,
          94108086667844986046802106544375316173742538919949485639896613738390948)


k_alice = Point(x=155781055760279718382374741001148850818103179141959728567110540865590463, y=73794785561346677848810778233901832813072697504335306937799336126503714)
k_bob = Point(x=171226959585314864221294077932510094779925634276949970785138593200069419, y=54353971839516652938533335476115503436865545966356461292708042305317630)
e_flag = {'iv': '64bc75c8b38017e1397c46f85d4e332b', 'encrypted_flag': '13e4d200708b786d8f7c3bd2dc5de0201f0d7879192e6603d7c5d6b963e1df2943e3ff75f7fda9c30a92171bbbc5acbf'}


The code line in source.py that made me suspicious was:
```python
    assert (A.x**2 - D*A.y**2) % p == 1
```
and from that everyhing made sense - the curve is not an **elliptic** curve, but an **ellipse**.
The curve is defined by the relation:
$$x^2 - D*y^2 = 1 \pmod p$$
So we can hope that for the case of an ellipse the scalar multiplication is easly reversible.

We'll look at - https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.66.8688&rep=rep1&type=pdf

Section 3 in the paper describes a way to create an isomorphism between the group of points on the curve and the group of integers modulo $p -\chi(D)$ where:
$$ \chi(D) = \begin{cases} 0 & \text{if } D=0 \\
                           1 & \text{if } D \text{ is quadratic residue modulo } p\\
                           -1 & \text{otherwise} \end{cases}$$

In [ ]:
chi_D = pow(D, (p-1)//2, p)
chi_D

In that case the paper tells us to find the sqrt of $D$:

In [ ]:
sqrtD = int(math.sqrt(D)) #we are lucky as D=23**2 
sqrtD

from which the following isomorphism is created:
$$ (x,y) \mapsto x - \sqrt{D}y$$

In [ ]:
point_iso = lambda P: (P.x - sqrtD*P.y) % p
G_iso = point_iso(G)
k_alice_iso = point_iso(k_alice)
k_bob_iso = point_iso(k_bob)

In [ ]:
G_iso, k_alice_iso, k_bob_iso

Now to solve the equation k_alice_iso**x = G_iso mod p we will use sage math to find the discrete logarithm. To do this will run the following lines in https://sagecell.sagemath.org/

```python
p=173754216895752892448109692432341061254596347285717132408796456167143559
Zp = Zmod(p)
G_iso = Zp(22288939911082159892714073512476247408798262113701860676893915460297072)
k_alice_iso = Zp(123713638361497775154772936433567841582759137820954323280958698315814611)
discrete_log(G_iso, k_alice_iso, p-1, operation='*')
```

The answer we get is
$$x = 85659350935432836744105386596354218808820976897571304400328185351959247$$

In [ ]:
#validate
na = 85659350935432836744105386596354218808820976897571304400328185351959247
scalar_multiplication(G, na) == k_alice

In [ ]:
iv = bytes.fromhex(e_flag['iv'])
encrypted = bytes.fromhex(e_flag['encrypted_flag'])

shared_secret = scalar_multiplication(k_bob, na).x #taken from "gen_shared_secret" in source.py

key = sha1(str(shared_secret).encode('ascii')).digest()[:16]
cipher = AES.new(key, AES.MODE_CBC, iv)
d = cipher.decrypt(encrypted)
d